In [1]:
import struct
import numpy

In [2]:
_format1 = ">4si"
_format2_small = ">iiiiiiBiii18s"

In [3]:
name = input("Enter name of the ROOT file - ")
if name.endswith(".root") is False:
    name = name + ".root"
file = numpy.memmap(filename = name, dtype = numpy.uint8, mode = "w+", shape = (80618,))
#In the case of a 1D array, the shape is the size of the file
#In ROOT files, the fEND corresponds to the size of the file
#fEND for ~/Downloads/uproot/tests/samples/sample-6.14.00-uncompressed.root is 80618

In [4]:
fVersion = 61400
begin = numpy.frombuffer(struct.pack(_format1, b"root", fVersion), dtype=numpy.uint8)
count = len(begin)
file[:count] = begin

In [6]:
#constants for file sample-6.14.00-uncompressed.root
fBEGIN = 100
fEND = 80618
fSeekFree = 80542
fNbytesFree = 76
nfree = 1
fNbytesName = 100
fUnits = 4
fCompress = 100
fSeekInfo = 63150
fNbytesInfo = 17282
fUUID = b'\x00\x01\x11\xaf/\xd0t^\x11\xe8\x8a\x9b\x01\x00\x00\x7f\xbe\xef'
start = numpy.frombuffer(struct.pack(_format2_small, fBEGIN, fEND, fSeekFree, fNbytesFree, nfree, fNbytesName, fUnits, fCompress, fSeekInfo, fNbytesInfo, fUUID), dtype=numpy.uint8)
file[count:(count + len(start))] = start
count = count + len(start)

In [7]:
#Test - See bytes read of copied file
!hexdump -C rootpy.root | head

00000000  72 6f 6f 74 00 00 ef d8  00 00 00 64 00 01 3a ea  |root.......d..:.|
00000010  00 01 3a 9e 00 00 00 4c  00 00 00 01 00 00 00 64  |..:....L.......d|
00000020  04 00 00 00 64 00 00 f6  ae 00 00 43 82 00 01 11  |....d......C....|
00000030  af 2f d0 74 5e 11 e8 8a  9b 01 00 00 7f be ef 00  |./.t^...........|
00000040  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
00013ae0  00 00 00 00 00 00 00 00  00 00                    |..........|
00013aea


In [10]:
#Test - See bytes read of target file
!hexdump -C tests/samples/sample-6.14.00-uncompressed.root | head

00000000  72 6f 6f 74 00 00 ef d8  00 00 00 64 00 01 3a ea  |root.......d..:.|
00000010  00 01 3a 9e 00 00 00 4c  00 00 00 01 00 00 00 64  |..:....L.......d|
00000020  04 00 00 00 64 00 00 f6  ae 00 00 43 82 00 01 11  |....d......C....|
00000030  af 2f d0 74 5e 11 e8 8a  9b 01 00 00 7f be ef 00  |./.t^...........|
00000040  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
00000060  00 00 00 00 00 00 00 a0  00 04 00 00 00 5e 5d a8  |.............^].|
00000070  9b a8 00 42 00 01 00 00  00 64 00 00 00 00 05 54  |...B.....d.....T|
00000080  46 69 6c 65 20 73 61 6d  70 6c 65 2d 36 2e 31 34  |File sample-6.14|
00000090  2e 30 30 2d 75 6e 63 6f  6d 70 72 65 73 73 65 64  |.00-uncompressed|


In [9]:
#Test - uproot is reading it correctly
import uproot
uproot.open('~/Downloads/uproot/rootpy.root')

IndexError: indexes 80618:80621 are beyond the end of data source '/home/chronos/user/Downloads/uproot/rootpy.root'

In [ ]:
#Test - Check size of file
import os
statinfo_target = os.stat('tests/samples/sample-6.14.00-uncompressed.root')
print ("Size of target file is ", statinfo_target.st_size)
statinfo_copy = os.stat('rootpy.root')
print ("size of written file is ", statinfo_copy.st_size)